In [1]:
import dreadnode as dn
from rich import print

dn.configure(
    server="http://localhost:8000",
    token="WNpw3Mz2saiyZ_SEM7Xl8qC1fhvqUCvZ"
)

Dreadnode Configuration: (from explicit parameters)

Server: http://localhost:8000

Organization: Main Organization

Workspace: Personal Workspace

Project: Default

In [2]:
import pyarrow.compute as pc
from dreadnode import load_dataset


# Load from remote (dn:// protocol)
ds = load_dataset("../data/dotnet", version="latest")
print(ds)
print(f"Rows: {len(ds)}")
print(f"Schema: {ds.schema}")

# Access data
table = ds.to_table()
df = ds.to_pandas()

Dataset(
    ds=pyarrow.Table
package_name: string
----
package_name: 
[["ActiproSoftware.Shared.Wpf","ActiproSoftware.SyntaxEditor.Wpf","K2.Framework.SSO.Base-signature","Apsis.NuGet.Tw
itterizer","Apsis.NuGet.TNValidate",...,"XImage.OCR.Language.Kannada","XImage.OCR.Language.Latin","XImage.OCR.Langu
age.Malayalam","Citms.Common.Core","XImage.OCR.Language.Maltese"],["XImage.OCR.Language.Macedonian","XImage.OCR.Lan
guage.MiddleEnglish","XImage.OCR.Language.Luxembourgish","XImage.OCR.Language.Mongolian","XImage.OCR.Language.Marat
hi",...,"Watch.The.Equalizer.3.Online.Fullmovie.Free.On.Streamings.At.Home","SolanaWallet","BonsaiDevelopment.Proto
cols.BARP","Pirater-Facebook-Facilement-En-Deux-Minutes-Hack-2023-VVGQzp","coin-master-free-spin-link-new"]],
    name='dotnet',
    description=None,
    version='0.0.1',
    license=None,
    tags=[],
    format='parquet'
)

Rows: 135539

Schema: package_name: string

In [3]:
ds.save()

Hashing files: 100%|██████████| 3/3 [00:00<00:00, 1102.51it/s]


In [ ]:
ds.metadata

In [ ]:


client = DatasetStorage()

def clean_dataset(ds):
    """Remove duplicates and null values"""
    table = ds.to_table()
    
    # Remove nulls
    table = table.filter(pc.is_valid(pc.field("text")))
    
    # Remove duplicates (example)
    # table = table.drop_duplicates()
    
    ds.ds = table
    return ds

# Load, update, and save new version
updated = client.update_dataset(
    uri="myorg/my-dataset",
    version="1.0.0",  # Load this version
    update_fn=clean_dataset,
)

print(f"Updated to version: {updated.version}")

In [ ]:
from dreadnode import Dataset

# Load CSV
csv_ds = Dataset.from_path("/data/my-data.csv", format="csv")

# Load JSON
json_ds = Dataset.from_path("/data/my-data.jsonl", format="json")

# Load from pandas
import pandas as pd
df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})
pandas_ds = Dataset.from_pandas(df, metadata={
    "name": "my-dataset",
    "description": "From pandas",
})

# Save in different formats
pandas_ds.save("/output/parquet", format="parquet")
pandas_ds.save("/output/csv", format="csv")
pandas_ds.save("/output/arrow", format="arrow")

In [ ]:
client = DatasetStorage()

# List cached datasets
datasets = client.list_datasets(cache_only=True)
for ds in datasets:
    print(f"{ds.name}@{ds.version} - {ds.description}")

# Search
results = client.search_datasets(
    name_pattern="prompts",
    tags=["nlp"],
)

# Get cache info
info = client.get_cache_info()
print(f"Cache size: {info['size_gb']:.2f} GB")
print(f"Datasets: {info['dataset_count']}")

# Remove specific version
client.remove_dataset("org/dataset", version="1.0.0")

# Remove all versions
client.remove_dataset("org/dataset")

# Clear entire cache (requires confirmation)
client.clear_cache(confirm=True)